In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["NCCL_IB_DISABLE"] = "1"
os.environ["NCCL_P2P_DISABLE"] = "1"

In [2]:
import torch
from datasets import load_dataset, load_metric,concatenate_datasets
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments,AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
from torch.utils.data import DataLoader
from tqdm import tqdm
import evaluate
import numpy as np
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score
from datasets import Dataset, DatasetDict, load_dataset, load_metric
import torch
import random

/media/.venv/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/media/.venv/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
from datasets import Dataset, DatasetDict, load_dataset, load_metric
from transformers import T5Tokenizer
from tqdm import tqdm
import json
import os

# Load dataset (example using CoLA)
dataset = load_dataset("glue", "cola")

# Tokenizer and metrics
tokenizer = T5Tokenizer.from_pretrained("t5-base")
metric = load_metric("glue", "cola")

# Prepare dataset
def preprocess_function(examples):
    input_ids = []
    attention_mask = []
    labels = []
    for i in tqdm(range(len(examples))):
        tokenized = tokenizer(examples["sentence"][i], truncation=True, max_length = 512)
        input_ids.append(tokenized["input_ids"])
        attention_mask.append(tokenized["attention_mask"])
        labels.append(examples["label"][i])
    return {"input_ids": input_ids, "attention_mask": attention_mask, "label": labels}

# Convert to Dataset object and preprocess
train_dataset = Dataset.from_dict(preprocess_function(dataset['train']))
eval_dataset = Dataset.from_dict(preprocess_function(dataset["validation"]))
test_dataset = Dataset.from_dict(preprocess_function(dataset["test"]))



You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/tmp/ipykernel_2642779/3068275542.py:12: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "cola")
/media/.venv/lib/python3.11/site-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be exe

In [4]:
len(train_dataset)

8551

In [5]:
num_epochs = 40
concatenated_data = concatenate_datasets([train_dataset, train_dataset])
for i in tqdm(range(1,num_epochs-1)):
    concatenated_data = concatenate_datasets([concatenated_data, train_dataset])
train_dataset = concatenated_data.shuffle(seed=42)  
print(f"Number of rows {len(train_dataset)}")

100%|█████████████████████████████████████████████████████████████████████████████████| 38/38 [00:00<00:00, 1159.84it/s]

Number of rows 342040


In [6]:
set_batch_size = 128
num_rows = len(train_dataset)

# Round down to the nearest hundred
rounded_num_rows = (num_rows // set_batch_size) * set_batch_size

# Truncate the dataset
train_dataset = train_dataset.select(range(rounded_num_rows))

In [7]:
# Create a DatasetDict
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": eval_dataset,
    "test": test_dataset
})

# Save the dataset to disk
save_path = "./preprocessed_cola_dataset"
dataset_dict.save_to_disk(save_path)

# Save preprocessing args to a json file
preprocessing_args = {
    "sequence_length": 512,
    "tokenizer": "t5-base"
}

with open(os.path.join(save_path, "args.json"), "w") as f:
    json.dump(preprocessing_args, f)

print(f"Dataset and preprocessing arguments saved to {save_path}")


Saving the dataset (0/1 shards):   0%|          | 0/342016 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1043 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/1063 [00:00<?, ? examples/s]

Dataset and preprocessing arguments saved to ./preprocessed_cola_dataset


In [8]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [9]:
print(f"Number of rows {len(train_dataset)}")
print(f"Number of steps {len(train_dataset)/set_batch_size}")

Number of rows 342016
Number of steps 2672.0


In [10]:
!torchrun --nproc_per_node=1 torchrun_main.py \
    --model_name_or_path t5-base \
    --dataset_path ./preprocessed_cola_dataset \
    --batch_size 128 \
    --lr 3e-4 \
    --max_length 512 \
    --weight_decay 0.01\
    --num_training_steps 2720 \
    --save_every 640\
    --eval_every 320 \
    --warmup_steps 40 \
    --seed 42\
    --tags relora_t5_base \
    --workers 4 \
    --use_double_quant False \
    --use_peft True \
    --cycle_length 340 \
    --restart_warmup_steps 10 \
    --scheduler cosine_restarts \
    --reset_optimizer_on_relora False \
    --relora 340\
    --lora_r 64\
    --optimizer_magnitude_pruning 0.9


/home/delyanh/Projects/ReLoRa/torchrun_main.py:153: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "cola")
/media/.venv/lib/python3.11/site-packages/datasets/load.py:759: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.2/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
2024-07-31 21:35:32.905 | INFO     | __main__:main:379 - Global rank 0, local rank 0, device: 0
2024-07-31 21:35:32.906 | INFO     | __main__:main:383 - Process group initialized
wandb: Curr

In [11]:
from peft_pretraining.relora import ReLoRaModel 
model = ReLoRaModel.from_pretrained("checkpoint/valiant-breeze-149/model_1920")

In [13]:
model.to("cuda")

ReLoRaModel(
  (wrapped_model): T5ForSequenceClassification(
    (transformer): T5Model(
      (shared): Embedding(32128, 768)
      (encoder): T5Stack(
        (embed_tokens): Embedding(32128, 768)
        (block): ModuleList(
          (0): T5Block(
            (layer): ModuleList(
              (0): T5LayerSelfAttention(
                (SelfAttention): T5Attention(
                  (q): ReLoRaLinear(
                    (lora_dropout): Dropout(p=0.1, inplace=False)
                    (lora_A): Linear(in_features=768, out_features=64, bias=False)
                    (lora_B): Linear(in_features=64, out_features=768, bias=False)
                  )
                  (k): ReLoRaLinear(
                    (lora_dropout): Dropout(p=0.1, inplace=False)
                    (lora_A): Linear(in_features=768, out_features=64, bias=False)
                    (lora_B): Linear(in_features=64, out_features=768, bias=False)
                  )
                  (v): ReLoRaLinear(
             

In [14]:
model.eval()
all_predictions = []
all_labels = []

for i in tqdm(range(len(eval_dataset))):
   #print(eval_dataset[i])
   inputs  = eval_dataset[i]
   #print(input)
   output = eval_dataset[i]["label"]

   model.eval()
# Forward pass to get logits (predictions)
   with torch.no_grad():

     outputs = model(input_ids = torch.tensor(inputs["input_ids"]).unsqueeze(0).to("cuda"), attention_mask = torch.tensor(inputs["attention_mask"]).unsqueeze(0).to("cuda"))
     logits = outputs.logits

# Convert logits to probabilities
   probabilities = torch.softmax(logits, dim=-1)

# Get predicted class label
   predicted_class = torch.argmax(logits, dim=-1).item()

   all_predictions.append(predicted_class)
   all_labels.append(output)

100%|███████████████████████████████████████████████████████████████████████████████| 1043/1043 [00:15<00:00, 65.51it/s]


In [15]:
results = metric.compute(predictions=all_predictions, references=all_labels)
print(results)

{'matthews_correlation': 0.5554433905906734}
